# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

---
## Step 0: Load The Data

In [1]:
import time
import pickle
import numpy as np
import tensorflow as tf
import csv
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.optimizers import Adam, SGD
from keras.layers import Input, Flatten, Dense, Conv2D, Add, Lambda, Cropping2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout, Concatenate
from keras.models import Model

import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

# Load pickled data
training_file = './traffic-signs-data/train.p'
validation_file= './traffic-signs-data/valid.p'
testing_file = './traffic-signs-data/test.p'

train_dict = pickle.load(open(training_file, mode='rb') )
valid_dict = pickle.load(open(validation_file, mode='rb'))
test_dict  = pickle.load(open(testing_file, mode='rb'))

X_train, y_train = train_dict['features'], train_dict['labels']
X_valid, y_valid = valid_dict['features'], valid_dict['labels']
X_test, y_test   = test_dict['features'], test_dict['labels']

Using TensorFlow backend.


In [2]:
#*******************************************************************************#
# Sameer Pawar, Oct-2018                                                                  #
# LeNet implementation in TensorFlow                              
# To Do:
# 1. No data preprocessing or augmentation
# 1. accept a different initialization function
# 1. accept different activation function
# 1. Loss: only support cross-entropy, no l_2 regularizations
# 1. 

"""
1. Had to add inits for initializer, batc_norm, optimizer
1. had to remove shape from biases when values are specified
1. had to provide none for bool placeholder.
1. 
"""

# Usage:
# 1. Define class with n_classes
# 1. Call compile to set opitmizers etc
# 1. Call fit with X_train, y_train, X_valid, y_ valid
# 1. Call get_fit_results_per_epoch
#*******************************************************************************#
import tensorflow as tf
from tensorflow.contrib.layers import flatten, batch_norm
import numpy as np
from sklearn.model_selection import train_test_split

class LeNet:

    def __init__(self, n_classes):        
        self.n_classes = n_classes        
        self.initializer = tf.contrib.layers.xavier_initializer() 
        self.activation_fun = 'relu'
        self.batch_norm = True
        self.optimizer = tf.train.AdamOptimizer()
        self.layers = {}

        # Define weights and biases for the graph
        self.weights = {
        'w_conv_0': tf.get_variable("w_conv_0", shape = [1, 1, 3, 1],       initializer = self.initializer),
        'w_conv_1': tf.get_variable("w_conv_1", shape = [5, 5, 1, 6],       initializer = self.initializer),
        'w_conv_2': tf.get_variable("w_conv_2", shape = [5, 5, 6, 16],      initializer = self.initializer),
        'w_fc_3':   tf.get_variable("w_fc_3",   shape = [400, 120],         initializer = self.initializer),
        'w_fc_4':   tf.get_variable("w_fc_4",   shape = [120, 84],          initializer = self.initializer),
        'w_fc_5':   tf.get_variable("w_fc_5",  shape = [84, self.n_classes],initializer = self.initializer)
        }

        self.biases = {
        'b_conv_0': tf.get_variable("b_conv_0", initializer = tf.zeros(1)),
        'b_conv_1': tf.get_variable("b_conv_1", initializer = tf.zeros(6)),
        'b_conv_2': tf.get_variable("b_conv_2", initializer = tf.zeros(16)),
        'b_fc_3':   tf.get_variable("b_fc_3",   initializer = tf.zeros(120)),
        'b_fc_4':   tf.get_variable("b_fc_4",   initializer = tf.zeros(84)),
        'b_fc_5':   tf.get_variable("b_fc_5", initializer = tf.zeros(self.n_classes))
        }
        
        self.keep_probabilities = np.ones(len(self.weights))

        self.x = tf.placeholder(tf.float32, (None, 32, 32, 3))
        self.y = tf.placeholder(tf.int32, (None))
        self.training = tf.placeholder(tf.bool, (None))

        #********************************************************************************************
        # Define compute nodes in a graph
        #********************************************************************************************
        self.one_hot_y          = tf.one_hot(self.y, self.n_classes)
        self.logits             = self.get_logits(self.x, self.training)
        self.cross_entropy      = tf.nn.softmax_cross_entropy_with_logits(labels=self.one_hot_y, logits=self.logits)
        self.loss_operation     = tf.reduce_mean(self.cross_entropy)    
        self.training_operation = self.optimizer.minimize(self.loss_operation)
        self.prediction         = tf.argmax(self.logits, 1)
        #********************************************************************************************
    
    def init_dropout_probabilities(self, dropout_probabilities):        
        if dropout_probabilities is not None: 
            if 'keep_conv_0' in dropout_probabilities:
                self.keep_probabilities[1] = dropout_probabilities['keep_conv_0']
            if 'keep_conv_1' in dropout_probabilities:
                self.keep_probabilities[2] = dropout_probabilities['keep_conv_1']
            if 'keep_conv_2' in dropout_probabilities:
                self.keep_probabilities[3] = dropout_probabilities['keep_conv_2']
            if 'keep_fc_3' in dropout_probabilities:
                self.keep_probabilities[4] = dropout_probabilities['keep_fc_3']
            if 'keep_fc_4' in dropout_probabilities:
                self.keep_probabilities[5] = dropout_probabilities['keep_fc_4']
        return

    def one_hot_encoding(self, labels):
        b = np.zeros((len(labels), self.n_classes))
        b[np.arange(len(labels)), labels] = 1
        return b

    def maxpool2d(self, x, k=2):
        return tf.nn.max_pool(
            x,
            ksize=[1, k, k, 1],
            strides=[1, k, k, 1],
            padding='SAME')

    def activation(self, x):    
        if self.activation_fun == 'relu':
            output = tf.nn.relu(x)
        elif self.activation_fun == 'elu':
            output = tf.nn.elu(x)
        elif self.activation_fun == 'lrelu':
            output = tf.nn.leaky_relu(x, alpha = 0.1)
        else:
            # default activation is 'linear'
            output = x

        if self.batch_norm:
            return batch_norm(output)
        else:
            return output

    def conv2d(self, x, W, b, strides=1):
        return tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID') + b

    def set_batch_norm(self, batch_norm):
        self.batch_norm = batch_norm

    def get_logits(self, x, training = True):
        # conv-Layer 1 - 32*32*3 -> 32*32*1
        # maps 3 color channels to one generalized grayscale channel
        self.layers['l_1'] = self.conv2d(x, W = self.weights['w_conv_0'], b = self.biases['b_conv_0'])
        if self.keep_probabilities[1] < 1 and training:
            self.layers['l_1'] = tf.nn.dropout(self.layers['l_1'], self.dropout_probabilities[1])        

        # conv-Layer 2 - 32*32*1 -> 28*28*6->14*14*6
        self.layers['l_2'] = self.conv2d(self.layers['l_1'], self.weights['w_conv_1'], self.biases['b_conv_1'])
        self.layers['l_2'] = self.maxpool2d(self.layers['l_2'])
        self.layers['l_2'] = self.activation(self.layers['l_2'])
        if self.keep_probabilities[2] < 1 and training:
            self.layers['l_2'] = tf.nn.dropout(self.layers['l_2'], self.dropout_probabilities[2])        

        # conv-Layer 3 - 14*14*6 -> 10*10*16->5*5*16
        self.layers['l_3'] = self.conv2d(self.layers['l_2'], self.weights['w_conv_2'], self.biases['b_conv_2'])
        self.layers['l_3'] = self.maxpool2d(self.layers['l_3'])
        self.layers['l_3'] = self.activation(self.layers['l_3'])
        if self.keep_probabilities[3] < 1 and training:
            self.layers['l_3'] = tf.nn.dropout(self.layers['l_3'], self.dropout_probabilities[3])        

        # FC-Layer 4 - 5*5*16-> 400->120
        self.layers['l_4'] = tf.matmul(flatten(self.layers['l_3']), self.weights['w_fc_3']) + self.biases['b_fc_3']
        self.layers['l_4'] = self.activation(self.layers['l_4'])
        if self.keep_probabilities[4] < 1 and training:
            self.layers['l_4'] = tf.nn.dropout(self.layers['l_4'], self.dropout_probabilities[4])        
        
        # FC-Layer 5 - 120-> 84
        self.layers['l_5'] = tf.matmul(self.layers['l_4'], self.weights['w_fc_4']) + self.biases['b_fc_4']
        self.layers['l_5'] = self.activation(self.layers['l_5'])
        if self.keep_probabilities[5] < 1 and training:
            self.layers['l_5'] = tf.nn.dropout(self.layers['l_5'], self.dropout_probabilities[5])

        # FC-Layer 6 - 84 -> 43 (n_classes)
        self.layers['l_6'] = tf.matmul(self.layers['l_5'], self.weights['w_fc_5']) + self.biases['b_fc_5']

        
        logits = self.layers['l_6']

        return logits

    def compile(self, optimizer = None, loss=None, metrics = None):
        """
        #************************************************************************#
            - optimizer: String (name of optimizer) or optimizer instance.
            - loss: String (name of objective function) or objective function.
            - metrics:  List of metrics to be evaluated by the model during training 
            and testing. Typically you will use  metrics=['accuracy']. 
            Also can pass multiple metrics such as 'accuracy', 
            'Precision-recall-f1score', etc.
        #************************************************************************#            
        """
        if optimizer is None:
            self.optimizer = tf.train.AdamOptimizer()
        else:
            self.optimizer = optimizer
        
        if loss is None:
            self.loss = 'cross-entropy'
        else:
            self.loss = loss

        if metrics is None:
            self.metrics = 'accuracy'    
        else:
            self.metrics = metrics # dictionary of different metrics
           
    def fit(self, x=None, y=None, generator = None, batch_size=None, epochs=1, validation_split=0.0, validation_data=None, dropout_probabilities = None, batch_norm = True):
        """
        #************************************************************************#
            - x: input features
            - y: labels
            - generator: function that takes in a batch of data (x, y) and returns a batch of data of same size but with transformed/augmented data
            - batch_size
            - epochs
            - validation_split 
            - validation_data: dictionary {'features': x_valid, 'labels': y_valid}
            - dropout_probabilities: dictionary of Keep probabilities with key = 'keep_conv/fc_id'
            - batch_norm: Flag to enable or disable batch_norm.
            Sameer: 
                - add adaptive rate                
                - not using generator or data preprocessing
        #************************************************************************#
        """
        self.training_loss          = np.zeros(epochs)
        self.training_accuracy      = np.zeros(epochs)
        self.training_precision     = np.zeros((self.n_classes, epochs))
        self.training_recall        = np.zeros((self.n_classes, epochs))
        self.training_F1score       = np.zeros((self.n_classes, epochs))

        self.validation_loss        = np.zeros(epochs)
        self.validation_accuracy    = np.zeros(epochs)
        self.validation_precision   = np.zeros((self.n_classes, epochs))
        self.validation_recall      = np.zeros((self.n_classes, epochs))
        self.validation_F1score     = np.zeros((self.n_classes, epochs))


        # Get training and validation data
        if validation_split > 0:
            x, X_valid, y, y_valid = train_test_split(x, y, train_size=1-validation_split)
        
        if validation_data is not None:
            X_valid, y_valid = validation_data['features'], validation_data['labels']


        # Read the keep probabilities from dictionary into variables.
        self.init_dropout_probabilities(dropout_probabilities)
        self.set_batch_norm(batch_norm)

        with tf.Session() as sess:      
            sess.run(tf.global_variables_initializer())
            for i in range(epochs):          
                print("running EPOCH {}".format(i+1))
                x, y = shuffle(x, y)
                for offset in range(0, x.shape[0], batch_size):
                    end = np.minimum(offset + batch_size, x.shape[0])
                    batch_x, batch_y = x[offset:end], y[offset:end]                    
                    sess.run(self.training_operation, feed_dict={self.x: batch_x, self.y: batch_y, self.training: True})
            
                # Compute the metrics for each epoch and log the results
                training_results   = self.evaluate(x, y)
                validation_results = self.evaluate(X_valid, y_valid)
                
                print("training accuracy is {}".format(training_results['accuracy']))
                self.training_accuracy[i]    = training_results['accuracy']
                self.training_loss[i]        = training_results['loss']        
                self.training_precision[:,i] = training_results['precision']
                self.training_recall[:,i]    = training_results['recall']
                self.training_F1score[:,i]   = training_results['F1score']
            
                self.validation_accuracy[i]    = validation_results['accuracy']
                self.validation_loss[i]        = validation_results['loss']        
                self.validation_precision[:,i] = validation_results['precision']
                self.validation_recall[:,i]    = validation_results['recall']
                self.validation_F1score[:,i]   = validation_results['F1score']    
        return

    def predict(self, x, top_k = 1):
        batch_size = 1024
        num_examples  = x.shape[0]
        sess = tf.get_default_session()
        prediction_result = np.ones((top_k, num_examples), dtype=np.float32)*(-1)
        for offset in range(0, num_examples, batch_size):
            end = np.minimum(offset + batch_size, num_examples)
            batch_x = x[offset:end]
            predicted_logits = sess.run(self.logits, feed_dict = {self.x: batch_x, self.training: False})
            prediction_result[:,offset:end] = np.argsort(predicted_logits, axis=0)[::-1][0:top_k+1,:]

        return prediction_result
   
    def evaluate(self, x=None, y=None):    
        batch_size = 1024
        num_examples  = x.shape[0]
        sess = tf.get_default_session()
        total_TP = np.zeros(self.n_classes, dtype=np.float32)
        total_FP = np.zeros(self.n_classes, dtype=np.float32)
        total_FN = np.zeros(self.n_classes, dtype=np.float32)    
        total_correct_predictions = 0
        total_loss = 0        
        error_list = []
        for offset in range(0, num_examples, batch_size):
            end = np.minimum(offset + batch_size, num_examples)
            batch_x, batch_y = x[offset:end], y[offset:end]
            predicted_labels, batch_loss = sess.run([self.prediction, self.loss_operation], 
                                                    feed_dict = {self.x: batch_x, 
                                                            self.y: batch_y,
                                                            self.training: False}
                                                    )
                       
            actuals      = self.one_hot_encoding(batch_y)        
            predictions  = self.one_hot_encoding(predicted_labels)
            total_TP += np.count_nonzero(predictions * actuals, axis=0)
            total_FP += np.count_nonzero(predictions * (actuals - 1), axis=0)
            total_FN += np.count_nonzero((predictions - 1) * actuals, axis=0)     
        
            
            total_correct_predictions += np.sum(np.equal(predicted_labels, batch_y))
            total_loss                += (batch_loss * batch_x.shape[0])

            predictions = np.array(predicted_labels).flatten()
            actuals     = np.array(batch_y).flatten()
            error_idx   = np.argwhere(np.not_equal(predictions, actuals)).ravel() + offset
            error_list.append(error_idx)        
        
        
        error_list = np.concatenate(error_list)
        accuracy  = total_correct_predictions/num_examples
        loss      = total_loss/num_examples
        precision = total_TP/(total_TP + total_FP + 1e-6)
        recall    = total_TP/(total_TP + total_FN + 1e-6)
        F1score   = 2 * precision * recall / (precision + recall + 1e-6)
        return {"loss": loss,
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "F1score": F1score,
                "error_list": error_list
            }

    def get_fit_results_per_epoch(self):
        return {'training_loss': self.training_loss, 
                'training_accuracy': self.training_accuracy,
                'validation_loss': self.validation_loss,
                'validation_accuracy': self.validation_accuracy
        }

# Train the Model


In [3]:
model = LeNet(43)
model.compile()
model.fit(x = X_train, y = y_train, batch_size = 64, 
          epochs = 5, 
          validation_data = {'features': X_valid, 'labels': y_valid}
         )
results = model.get_fit_results_per_epoch()
training_loss = results['training_loss']
training_accuracy = results['training_accuracy']
validation_accuracy = results['validation_accuracy']
validation_loss = results['validation_loss']
print(training_accuracy)
print(validation_accuracy)



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

running EPOCH 1
training accuracy is 0.9595390672145752
running EPOCH 2
training accuracy is 0.9855168251961264
running EPOCH 3
training accuracy is 0.9929883042616167
running EPOCH 4
training accuracy is 0.9972125635794132
running EPOCH 5
training accuracy is 0.99643667921492
[0.95953907 0.98551683 0.9929883  0.99721256 0.99643668]
[0.75419501 0.79886621 0.81882086 0.81882086 0.82789116]
